In [84]:
import lxml
import selenium
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set(rc={'figure.figsize':(20,10)})
sns.set_theme()

In [85]:
races=pd.read_csv("race.csv")
results=pd.read_csv("results.csv")
constructor_standings=pd.read_csv("constructor_standings.csv")
driver_standings=pd.read_csv("driver_standings.csv")
qualifying_results=pd.read_csv("qualifying_results.csv")
weather=pd.read_csv("weather.csv")

In [86]:
races=races.drop(columns="Unnamed: 0")
results=results.drop(columns="Unnamed: 0")
constructor_standings=constructor_standings.drop(columns="Unnamed: 0")
driver_standings=driver_standings.drop(columns="Unnamed: 0")
qualifying_results=qualifying_results.drop(columns="Unnamed: 0")
weather=weather.drop(columns="Unnamed: 0")

In [87]:
qualifying_results["driver_name"]=qualifying_results["driver_name"].str.split()

In [88]:
# on modifie la colonne "driver_name" de qualifying_results pour faciliter le merge
taille=len(qualifying_results)
for i in range(taille):
    qualifying_results.iloc[i,1]=qualifying_results.iloc[i,1][1]
    
qualifying_results["driver_name"]

0            Berger
1           Martini
2                de
3             Alesi
4             Senna
            ...    
12069       Russell
12070    Fittipaldi
12071        Latifi
12072         Perez
12073     Magnussen
Name: driver_name, Length: 12074, dtype: object

In [89]:
qualifying_results.rename(columns={'driver_name': 'driver'}, inplace=True)
qualifying_results["driver"]=qualifying_results["driver"].str.lower()

In [90]:
## on merge constructor_standings sur results 

results_2=results.merge(how="left",on=["season","round","constructor"],right=constructor_standings)
results_2.head()
results_3=results_2.merge(how="left",on=["season","round","driver"],right=driver_standings)
results_3.head(30)
results_4=results_3.merge(how="left",on=["season","round","driver"],right=qualifying_results)
results_5=results_4.merge(how="left",on=["season","round","circuit_id"],right=weather)

In [91]:
results_5=results_5.drop(columns="grid_y")
results_5.rename(columns={'grid_x': 'grid'}, inplace=True)
df_merged=results_5.copy()

In [92]:
df_merged_na=df_merged.isna().copy()

In [93]:
df_merged.drop(columns='time', inplace=True)

In [94]:
var= ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins' , 'constructor_standings_pos']
for variable in var:
    df_merged[variable].fillna(0, inplace = True)
    df_merged[variable] = df_merged[variable].map(lambda x: int(x))
df_merged.dropna(inplace = True )

In [95]:
# Ajout de l'âge des pilote
df_merged["date_of_birth"]=pd.to_datetime(df_merged["date_of_birth"])
df_merged["season"]=pd.to_datetime(df_merged["season"], format="%Y") # conversion en format date
df_merged["age"]=round(((df_merged["season"]-df_merged['date_of_birth']).dt.days)/365) #On détermine l'âge du pilote

In [96]:
df_merged['age']

0        30.0
1        26.0
2        32.0
3        37.0
4        27.0
         ... 
12391    22.0
12392    33.0
12393    22.0
12394    26.0
12395    25.0
Name: age, Length: 10004, dtype: float64

In [97]:
for col in ['weather_warm', 'weather_cold','weather_dry', 'weather_wet', 'weather_cloudy']:
    df_merged[col] = df_merged[col].map(lambda x: bool(x))

In [98]:
df_merged['qualifying_time'] = df_merged.qualifying_time.map(lambda x: 0 if str(x) == '00.000' else(float(str(x).split(':')[1]) + (60 * float(str(x).split(':')[0])) if len(x.split(':'))>1 else 0))

In [99]:
df_merged = df_merged[df_merged['qualifying_time'] != 0] # On enlève les valeurs manquantes
df_merged.sort_values(['season', 'round', 'grid'], inplace = True)
df_merged['qualifying_time_diff'] = df_merged.groupby(['season', 'round']).qualifying_time.diff()
df_merged['qualifying_time'] = df_merged.groupby(['season', 
                                                'round']).qualifying_time_diff.cumsum().fillna(0)
df_merged.drop('qualifying_time_diff', axis = 1, inplace = True)

In [100]:
df_merged['qualifying_time']

17       0.000
6        0.067
1        0.744
0        0.767
3        1.198
         ...  
12391    0.794
12392    1.360
12394    2.977
12393    2.774
12395    3.172
Name: qualifying_time, Length: 9984, dtype: float64

In [101]:
df_dum = pd.get_dummies(df_merged, columns = ['circuit_id', 'nationality', 'constructor'] )

for col in df_dum.columns:
    if 'nationality' in col and df_dum[col].sum() < 140: # critère arbitraire pour enlever les colonnes non significatives (pris chez la fille)
        df_dum.drop(columns=col,inplace=True)
        
    elif 'constructor' in col and df_dum[col].sum() < 140: # critère arbitraire pour enlever les colonnes non significatives
        df_dum.drop(columns=col,inplace=True)
        
    elif 'circuit_id' in col and df_dum[col].sum() < 70:  # critère arbitraire pour enlever les colonnes non significatives
        df_dum.drop(columns=col,inplace=True)

In [102]:
del df_dum['date_of_birth']
del df_dum['status']
del df_dum['car']
del df_dum['weather']

In [103]:
df_dum['season']=pd.to_datetime(df_dum['season'])
df_dum['season'] = df_dum['season'].dt.year

In [104]:
df_dum.to_csv('final_df_merged.csv', index = False)